In [1]:
import random
from pathlib import Path
import pandas as pd

from TELF.pipeline.blocks.base_block import AnimalBlock
from TELF.pipeline.blocks.data_bundle import DataBundle, SAVE_DIR_BUNDLE_KEY
from TELF.pipeline import BlockManager

class RandomCheckpointBlock(AnimalBlock):
    def __init__(self, tag: str, verbose: bool = False, load_checkpoint: bool = True):
        super().__init__(
            needs=(), provides=("df",), tag=tag,
            verbose=verbose, load_checkpoint=load_checkpoint
        )

    def run(self, bundle: DataBundle) -> DataBundle:
        n = random.randint(0, 999)
        df = pd.DataFrame({"value": [n]})
        out_dir = Path(bundle[SAVE_DIR_BUNDLE_KEY]) / self.tag
        out_dir.mkdir(parents=True, exist_ok=True)
        csv_path = out_dir / "df.csv"
        df.to_csv(csv_path, index=False)
        self.register_checkpoint("df", csv_path)
        bundle[f"{self.tag}.df"] = df
        return bundle

def two_blocks_two_runs(flag):
    root = Path("example_results") / f"flag_{flag!s}"
    bundle = DataBundle({SAVE_DIR_BUNDLE_KEY: root})

    # Block 1: default load_checkpoint=True
    b1 = RandomCheckpointBlock(tag="RandomCP1", verbose=False)
    # Block 2: forced load_checkpoint=False
    b2 = RandomCheckpointBlock(tag="RandomCP2", verbose=False, load_checkpoint=False)

    mgr = BlockManager(
        blocks=[b1, b2],
        databundle=bundle,
        verbose=False,
        progress=False,
        capture_output=None,
        force_checkpoint=flag,
    )

    # First run
    out = mgr()
    v1_b1, v1_b2 = (
        out["RandomCP1.df"]["value"].iloc[0],
        out["RandomCP2.df"]["value"].iloc[0],
    )

    # Second run
    out = mgr()
    v2_b1, v2_b2 = (
        out["RandomCP1.df"]["value"].iloc[0],
        out["RandomCP2.df"]["value"].iloc[0],
    )

    return v1_b1, v1_b2, v2_b1, v2_b2



In [2]:
flags = (None, True, False)
results = {}

# ─── First loop: run tests ──────────────────────────────────────────
for flag in flags:
    results[flag] = two_blocks_two_runs(flag)

In [3]:

# ─── Second loop: print summaries ──────────────────────────────────
for flag in flags:
    v1_b1, v1_b2, v2_b1, v2_b2 = results[flag]

    print("\n" + "="*60)
    print(f"▶▶▶ manager.force_checkpoint = {flag!r}\n")

    # Block 1 summary
    print("Block 1 (RandomCP1, default load_checkpoint=True):")
    print(f"  • run 1 → {v1_b1}")
    print(f"  • run 2 → {v2_b1}")
    expect_b1 = "loaded" if flag is not False else "skipped"
    outcome_b1 = "match" if v1_b1 == v2_b1 else "differ"
    if (flag is True or (flag is None and expect_b1=="loaded") or (flag is False and expect_b1=="skipped")):
        correct = (expect_b1=="loaded" and outcome_b1=="match") or (expect_b1=="skipped" and outcome_b1=="differ")
    else:
        correct = False
    print(f"    ↳ values {outcome_b1} → expected to {expect_b1} checkpoint"
            f"{' ✅' if correct else ' ❌'}\n")

    # Block 2 summary
    print("Block 2 (RandomCP2, load_checkpoint=False):")
    print(f"  • run 1 → {v1_b2}")
    print(f"  • run 2 → {v2_b2}")
    # determine expected behavior for block2
    if flag is True:
        expect_b2 = "loaded"
    else:
        expect_b2 = "skipped"
    outcome_b2 = "match" if v1_b2 == v2_b2 else "differ"
    correct_b2 = (expect_b2=="loaded" and outcome_b2=="match") or (expect_b2=="skipped" and outcome_b2=="differ")
    print(f"    ↳ values {outcome_b2} → expected to {expect_b2} checkpoint"
            f"{' ✅' if correct_b2 else ' ❌'}")



▶▶▶ manager.force_checkpoint = None

Block 1 (RandomCP1, default load_checkpoint=True):
  • run 1 → 290
  • run 2 → 290
    ↳ values match → expected to loaded checkpoint ✅

Block 2 (RandomCP2, load_checkpoint=False):
  • run 1 → 281
  • run 2 → 838
    ↳ values differ → expected to skipped checkpoint ✅

▶▶▶ manager.force_checkpoint = True

Block 1 (RandomCP1, default load_checkpoint=True):
  • run 1 → 55
  • run 2 → 55
    ↳ values match → expected to loaded checkpoint ✅

Block 2 (RandomCP2, load_checkpoint=False):
  • run 1 → 453
  • run 2 → 453
    ↳ values match → expected to loaded checkpoint ✅

▶▶▶ manager.force_checkpoint = False

Block 1 (RandomCP1, default load_checkpoint=True):
  • run 1 → 855
  • run 2 → 577
    ↳ values differ → expected to skipped checkpoint ✅

Block 2 (RandomCP2, load_checkpoint=False):
  • run 1 → 358
  • run 2 → 493
    ↳ values differ → expected to skipped checkpoint ✅
